In [28]:
import pandas as pd
import pickle
from tqdm import tqdm
import time

In [21]:
from openai import OpenAI
client = OpenAI()

In [22]:
r = pd.read_csv('../generations/rmt_test.csv')
words = r['Word']
bfs_concepts = r['BFSConcepts']
pq_concepts = r['PQConcepts']

In [24]:
openai_generations = []

In [29]:
for i in tqdm(range(3,103)):
  word = words[i]
  cp = pq_concepts[i]
  msg = f"Can you create a riddle for the word: {word} which touches upon these concepts: {cp[:4]} ?"
  completion = client.chat.completions.create(
      model="gpt-3.5-turbo-1106",
      messages=[
        {"role": "user", "content": msg}
      ]
  )
  openai_generations.append(completion.choices[0].message.content)
  time.sleep(20)

100%|██████████| 100/100 [36:35<00:00, 21.95s/it]


In [38]:
openai_gen = {
    "Words":words[:103],
    "PQConcepts":pq_concepts[:103],
    "Generation":openai_generations
}

In [55]:
prompts = []
for i in tqdm(range(103,len(words))):
  word = words[i]
  cp = pq_concepts[i]
  msg = f"Can you create a riddle for the word: {word} which touches upon these concepts: {eval(cp)[:4]} ?"
  prompts.append(msg)

100%|██████████| 766/766 [00:00<00:00, 25713.60it/s]


In [60]:
for i in tqdm(range(39)):
      ip_prompts = prompts[20*i:20*(i+1)]
      completion = client.completions.create(
            model="gpt-3.5-turbo-instruct",
            prompt=ip_prompts,
            max_tokens=128
      )
      for resp in completion.choices:
            gen = resp.text
            openai_generations.append(gen)
      time.sleep(20)
      

100%|██████████| 39/39 [16:22<00:00, 25.20s/it]


In [63]:
openai_gen = {
    "Words":words,
    "PQConcepts":pq_concepts,
    "Generation":openai_generations
}

In [64]:
pd.DataFrame.from_dict(openai_gen).to_csv('gpt-turbo-gen-pq.csv',index=False)

In [ ]:
openai_generations.append(completion.choices[0].message.content)